# HDB Resale Price Preprocessing

## 1. Import dataset

In [63]:
import pandas as pd

# Load from local CSV file instead of API
file_path = "../data/raw/RawResalePrice.csv"
df = pd.read_csv(file_path)


# Select relevant columns and remove the others
columns_to_keep = [
    #"_id",
    "month",
    #"town",
    "flat_type",
    "block",
    "street_name",
    "storey_range",
    "floor_area_sqm",
    "flat_model",
    #"lease_commence_date",
    "remaining_lease",
    "resale_price"
]
df = df[columns_to_keep]

# Show cleaned DataFrame
print("Dataset Size: ", len(df))
print("->Examples:")
print(df.head())






Dataset Size:  204116
->Examples:
     month flat_type block        street_name storey_range  floor_area_sqm  \
0  2017-01    2 ROOM   406  ANG MO KIO AVE 10     10 TO 12            44.0   
1  2017-01    3 ROOM   108   ANG MO KIO AVE 4     01 TO 03            67.0   
2  2017-01    3 ROOM   602   ANG MO KIO AVE 5     01 TO 03            67.0   
3  2017-01    3 ROOM   465  ANG MO KIO AVE 10     04 TO 06            68.0   
4  2017-01    3 ROOM   601   ANG MO KIO AVE 5     01 TO 03            67.0   

       flat_model     remaining_lease  resale_price  
0        Improved  61 years 04 months      232000.0  
1  New Generation  60 years 07 months      250000.0  
2  New Generation  62 years 05 months      262000.0  
3  New Generation   62 years 01 month      265000.0  
4  New Generation  62 years 05 months      265000.0  


## 2. Cleaning Storey range and Remaining Lease


In [64]:
# Convert Storey range to average storey
def storey_to_avg(storey_range):
    try:
        low, high = map(int, storey_range.split(" TO "))
        return (low + high) // 2
    except:
        return None  

# Convert Remaining lease to months
def parse_remaining_lease(lease_str):
    try:
        parts = lease_str.split()
        years = int(parts[0])
        months = int(parts[2]) if "month" in lease_str else 0
        return years * 12 + months
    except:
        return None
    
# Apply changes to the DF
df["storey_avg"] = df["storey_range"].apply(storey_to_avg)
df["remaining_lease_months"] = df["remaining_lease"].apply(parse_remaining_lease)

# Drop original columns
df = df.drop(columns=["remaining_lease", "storey_range"])

# Check result
print(df.head())

     month flat_type block        street_name  floor_area_sqm      flat_model  \
0  2017-01    2 ROOM   406  ANG MO KIO AVE 10            44.0        Improved   
1  2017-01    3 ROOM   108   ANG MO KIO AVE 4            67.0  New Generation   
2  2017-01    3 ROOM   602   ANG MO KIO AVE 5            67.0  New Generation   
3  2017-01    3 ROOM   465  ANG MO KIO AVE 10            68.0  New Generation   
4  2017-01    3 ROOM   601   ANG MO KIO AVE 5            67.0  New Generation   

   resale_price  storey_avg  remaining_lease_months  
0      232000.0          11                     736  
1      250000.0           2                     727  
2      262000.0           2                     749  
3      265000.0           5                     745  
4      265000.0           2                     749  


## 3. Cleaning Address


In [65]:
# Combine block and street_name into one address string
df["address"] = df["block"].astype(str) + " " + df["street_name"]

# Drop the original columns
df = df.drop(columns=["block", "street_name"])

# Preview the new address column
print(df.head())


     month flat_type  floor_area_sqm      flat_model  resale_price  \
0  2017-01    2 ROOM            44.0        Improved      232000.0   
1  2017-01    3 ROOM            67.0  New Generation      250000.0   
2  2017-01    3 ROOM            67.0  New Generation      262000.0   
3  2017-01    3 ROOM            68.0  New Generation      265000.0   
4  2017-01    3 ROOM            67.0  New Generation      265000.0   

   storey_avg  remaining_lease_months                address  
0          11                     736  406 ANG MO KIO AVE 10  
1           2                     727   108 ANG MO KIO AVE 4  
2           2                     749   602 ANG MO KIO AVE 5  
3           5                     745  465 ANG MO KIO AVE 10  
4           2                     749   601 ANG MO KIO AVE 5  


## 4. Convert time

In [66]:

# Define start date
start_date = pd.to_datetime("2017-01")

# Convert month to datetime
df["month"] = pd.to_datetime(df["month"])

# Calculate time difference in months
df["time"] = ((df["month"].dt.year - start_date.year) * 12 +
              (df["month"].dt.month - start_date.month))

# Check result
print(df.sample(10))


            month  flat_type  floor_area_sqm         flat_model  resale_price  \
21231  2018-01-01     4 ROOM            90.0            Model A      348000.0   
45907  2019-03-01     4 ROOM            96.0  Premium Apartment      495000.0   
74692  2020-07-01     4 ROOM            92.0            Model A      410000.0   
171399 2024-05-01     5 ROOM           133.0            Model A      670000.0   
54875  2019-08-01     4 ROOM            91.0            Model A      690000.0   
94130  2021-03-01  EXECUTIVE           146.0         Maisonette      950000.0   
162440 2023-12-01     5 ROOM           111.0  Premium Apartment      615000.0   
117823 2022-01-01  EXECUTIVE           148.0         Maisonette      950000.0   
50377  2019-05-01     5 ROOM           121.0  Premium Apartment      423000.0   
123958 2022-12-01     4 ROOM            93.0            Model A      510000.0   

        storey_avg  remaining_lease_months                 address  time  
21231            2               

In [67]:
# Great! Months is not needed
df = df.drop(columns=["month"])

## 5. A few more for Flat type

In [68]:
flat_type_order = {
    "1 ROOM": 1,
    "2 ROOM": 2,
    "3 ROOM": 3,
    "4 ROOM": 4,
    "5 ROOM": 5,
    "EXECUTIVE": 6,
    "MULTI-GENERATION": 7
}

df["flat_type_encoded"] = df["flat_type"].map(flat_type_order)

df = df.drop(columns=["flat_type"])

print(df.head())

   floor_area_sqm      flat_model  resale_price  storey_avg  \
0            44.0        Improved      232000.0          11   
1            67.0  New Generation      250000.0           2   
2            67.0  New Generation      262000.0           2   
3            68.0  New Generation      265000.0           5   
4            67.0  New Generation      265000.0           2   

   remaining_lease_months                address  time  flat_type_encoded  
0                     736  406 ANG MO KIO AVE 10     0                  2  
1                     727   108 ANG MO KIO AVE 4     0                  3  
2                     749   602 ANG MO KIO AVE 5     0                  3  
3                     745  465 ANG MO KIO AVE 10     0                  3  
4                     749   601 ANG MO KIO AVE 5     0                  3  


## 6. Reorder the columns and output

In [69]:
# Reorder columns to look better :D
df = df[[
    "time",
    "address",
    "storey_avg",
    "floor_area_sqm",
    "flat_type_encoded",
    "flat_model",
    "remaining_lease_months",
    "resale_price"
]]

# Preview result
print(df.head())

# Save to CSV
output_path = "../data/processed/cleaned_resale_data.csv"
df.to_csv(output_path, index=False)

print("-" * 20)
print(f"Saved cleaned data to: {output_path}")

   time                address  storey_avg  floor_area_sqm  flat_type_encoded  \
0     0  406 ANG MO KIO AVE 10          11            44.0                  2   
1     0   108 ANG MO KIO AVE 4           2            67.0                  3   
2     0   602 ANG MO KIO AVE 5           2            67.0                  3   
3     0  465 ANG MO KIO AVE 10           5            68.0                  3   
4     0   601 ANG MO KIO AVE 5           2            67.0                  3   

       flat_model  remaining_lease_months  resale_price  
0        Improved                     736      232000.0  
1  New Generation                     727      250000.0  
2  New Generation                     749      262000.0  
3  New Generation                     745      265000.0  
4  New Generation                     749      265000.0  
--------------------
Saved cleaned data to: ../data/processed/cleaned_resale_data.csv


## 7. Dividing into test & train set

In [70]:
from sklearn.model_selection import train_test_split

# 80% train, 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

# Save train and test sets to model folder
train_df.to_csv("../data/processed/train.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)

print("Train and test sets saved to ../data/processed/")



Train set size: 163292
Test set size: 40824
Train and test sets saved to ../data/processed/


## 8. Add-on: Searching / Filtering function

In [71]:
def filter_data(df, **filters):
    """
    Filters the DataFrame based on:
    - Exact match: column=value
    - Range filter: min_col=value or max_col=value
    - Partial string match: contains_col='text'
    
    Prints the first 10 matching rows.
    """
    filtered_df = df.copy()
    
    for key, value in filters.items():
        if key.startswith("min_"):
            col = key[4:]
            filtered_df = filtered_df[filtered_df[col] >= value]
        elif key.startswith("max_"):
            col = key[4:]
            filtered_df = filtered_df[filtered_df[col] <= value]
        elif key.startswith("contains_"):
            col = key[9:]
            filtered_df = filtered_df[filtered_df[col].str.contains(value, case=False, na=False)]
        else:
            # exact match
            filtered_df = filtered_df[filtered_df[key] == value]

    if filtered_df.empty:
        print("No matching records found.")
    else:
        print(filtered_df.head(10))


In [ ]:
# Exact match
filter_data(df, time=12)

# Numeric range
filter_data(df, min_time=50, max_time=60)

# String contains
filter_data(df, contains_address="choa chu kang")

# Combine filters
filter_data(df, flat_type_encoded=4, min_floor_area_sqm=100, contains_address="yishun")


       time                address  storey_avg  floor_area_sqm  \
20509    12   314 ANG MO KIO AVE 3          11            44.0   
20510    12   109 ANG MO KIO AVE 4           2            67.0   
20511    12   150 ANG MO KIO AVE 5           2            68.0   
20512    12  559 ANG MO KIO AVE 10           2            68.0   
20513    12  461 ANG MO KIO AVE 10          11            68.0   
20514    12  466 ANG MO KIO AVE 10           5            67.0   
20515    12   152 ANG MO KIO AVE 5           5            68.0   
20516    12   348 ANG MO KIO AVE 3          11            73.0   
20517    12   503 ANG MO KIO AVE 5           2            68.0   
20518    12  557 ANG MO KIO AVE 10           5            68.0   

       flat_type_encoded      flat_model  remaining_lease_months  resale_price  
20509                  2        Improved                     709      250000.0  
20510                  3  New Generation                     715      240000.0  
20511                  3  New 

KeyError: 'flat_type'